# About
- **By**: Tsombou Christian
- **@** : tsombouchris@gmail.com
- **linkedIn**: https://www.linkedin.com/in/tsombouchris/

This Notebook performs an Scrapping of POSOCO website in order to obtain on Demand and Supply of Energy in Tamil Nadu in India from PDF periodical reports on the period July 2014 to May 2022

- **Data Source**: [POSOCO website](https://posoco.in/reports/monthly-reports/)
- **Data unit**: Demand and Supply of Energy is in MU - Million Unit (unit of energy) or in MW Mega-Watt
- **Reference**: we started the pdf scrapping with [tabula library](https://tabula-py.readthedocs.io/en/latest/) tabula is also [here](https://www.youtube.com/watch?v=tEFAFQXaOWw) but ended the job with [camelot library](https://camelot-py.readthedocs.io/en/master/) which we found more interesting for this task. We Used the [Selenium library](https://selenium-python.readthedocs.io/) for scraping the website


_Process:_

for each of the eleven urls of website pages with reports, we:
 - had to uncomments the urls only one at a time to do the scrapping
 - Scraped the page to obtain reports names, their link and other; this was done with selenium
 - Using Tabula or Camelot we scraped each PDF file for demand and supply data; this was done by identifying the page corresponding to the targeted data, and scrapped a table to gather data corresponding only to Tamil Nadu.

 _Result:_
 - A 130 entries and 14 Columns dataset store in a .csv file

 _Challenges:_

 Due to the fact that all reports were not having the same exact formating, we were at many ocasions obliged to localise the targeted data by opening the website and a monthly report and confirm the exact location of the data to scrap.

NB: This Notebook was produced within the context of The Omdena [Chennai Chapter project on Electricity Power Outage Analysis](https://github.com/OmdenaAI/chennai-india-power-outage)

### 1. Scrapping the website to obtain reports

In [4]:

# ##   Installing useful Libraries


# ! pip install tabula-py
# ! pip install "camelot-py[base]"
# ! pip install ghostscript

In [150]:
# Importing useful Libraries
from selenium import webdriver
import pandas as pd
import tabula
import ghostscript
import _tkinter
import camelot
import time

In [ ]:
# Building our empty dataset to append rows after scrapping 

df_ = pd.DataFrame(columns=['state', 'Requirement(MU/DAY)', 'Energy_met(MU/DAY)', 'Surplus(+)/Deficit(-)(MU/DAY)', 'Requirement(MU)',
 'Energy_met(MU)', 'Surplus(+)/Deficit(-)(MU)', '%Shortage(MU)', 'Requirement(MW)', 'Peak_Demand_Met5(MW)', 
 'Surplus(+)/Deficit(-)(MW)', '%Shortage(MW)', 'date_report', 'name_report'])

In [151]:
df_=pd.read_csv('loadschedding_TamilNadu.csv')

In [204]:
# Enumerating the target urls to be scraped one after the other by uncommenting them

# first url
url = 'https://posoco.in/reports/monthly-reports/monthly-reports-2022-23/'
# second url
# url = 'https://posoco.in/reports/monthly-reports/monthly-reports-2021-22/'
# third url
# url = 'https://posoco.in/reports/monthly-reports/monthly-reports-2020-21/'
# fourth url
# url ='https://posoco.in/reports/monthly-reports/monthly-reports-2019-20/'
# fifth url
# url = 'https://posoco.in/reports/monthly-reports/monthly-reports-2018-19/'
# sixth url
#url = 'https://posoco.in/reports/monthly-reports/monthly-reports-2017-18/'
# seventh url
# url = 'https://posoco.in/reports/monthly-reports/monthly-reports-2016-17/'
# eigth url
# url = 'https://posoco.in/reports/monthly-reports/monthly-reports-2015-16/'
# ninth url
# url = 'https://posoco.in/reports/monthly-reports/monthly-reports-2014-15/'
# tenth url
# url = 'https://posoco.in/reports/monthly-reports/monthly-reports-2013-14/'
# eleventh url
# url = 'https://posoco.in/reports/monthly-reports/monthly-reports-2012-13/'

In [205]:
# setting the webdriver with Chrome
driver = webdriver.Chrome()

In [206]:
# Access the webpage
driver.get(url)

In [207]:
#Performing the Scrapping after analysis of the webpage format

from selenium.webdriver.common.by import By
classes = ['odd', 'even'] # the report are either from the 'odd' or 'even' classes
rows = []                  # a list to contain our results
for clas in classes:
    monthly_reports = driver.find_elements(By.CLASS_NAME, clas)                  
    for report in monthly_reports:
        date_report = report.find_elements(By.CLASS_NAME, 'hidden-xs')[1].text
        name_report = report.find_element(By.TAG_NAME,'a').text
        link_report = report.find_element(By.TAG_NAME, 'a').get_attribute('href')
        rows.append([date_report, name_report, link_report])

rows

[['23-04-2019',
  'Monthly_Report_March_2019',
  'https://posoco.in/download/monthly_report_march_2019/?wpdmdl=22786'],
 ['22-02-2019',
  'Monthly Report January 2019',
  'https://posoco.in/download/monthly-report-january-2019/?wpdmdl=22031'],
 ['21-12-2018',
  'Monthly Report November 2018',
  'https://posoco.in/download/monthly-report-november-2018/?wpdmdl=20943'],
 ['23-10-2018',
  'Monthly Report September 2018',
  'https://posoco.in/download/monthly-report-september-2018/?wpdmdl=20135'],
 ['23-08-2018',
  'Monthly Report July 2018',
  'https://posoco.in/download/monthly-report-july-2018/?wpdmdl=19263'],
 ['22-06-2018',
  'Monthly Report May 2018',
  'https://posoco.in/download/monthly-report-may-2018/?wpdmdl=18367'],
 ['22-03-2019',
  'Monthly_report_February_2019',
  'https://posoco.in/download/monthly_report_february_2019/?wpdmdl=22125'],
 ['23-01-2019',
  'Monthly Report December 2018',
  'https://posoco.in/download/monthly-report-december-2018/?wpdmdl=21407'],
 ['22-11-2018',


In [208]:
# our output in tabular format
df = pd.DataFrame(rows, columns=['date_report', 'name_report', 'link_report'])
df

,date_report,name_report,link_report
0,23-04-2019,Monthly_Report_March_2019,https://posoco.in/download/monthly_report_marc...
1,22-02-2019,Monthly Report January 2019,https://posoco.in/download/monthly-report-janu...
2,21-12-2018,Monthly Report November 2018,https://posoco.in/download/monthly-report-nove...
3,23-10-2018,Monthly Report September 2018,https://posoco.in/download/monthly-report-sept...
4,23-08-2018,Monthly Report July 2018,https://posoco.in/download/monthly-report-july...
5,22-06-2018,Monthly Report May 2018,https://posoco.in/download/monthly-report-may-...
6,22-03-2019,Monthly_report_February_2019,https://posoco.in/download/monthly_report_febr...
7,23-01-2019,Monthly Report December 2018,https://posoco.in/download/monthly-report-dece...
8,22-11-2018,Monthly Report October 2018,https://posoco.in/download/monthly-report-octo...
9,23-09-2018,Monthly Report August 2018,https://posoco.in/download/monthly-report-augu...


### 2. Scrapping PDF files for demand and supply data

dfs = tabula.read_pdf(df.loc[1,'link_report'], lattice=True, pandas_options={"header": [0, 1]},  pages='29')
#tabula.convert_into(df_22_23.loc[0,'link_report'], "output.csv", output_format="csv", pages='29')
row = list(dfs[0].loc[27,1:]) + [df.iloc[1,0], df.iloc[1,1]]
row_dict = {df_.columns[i]:row[i] for i in range(len(row))}
df_ = df_.append(row_dict, ignore_index=True)

In [209]:
tables = camelot.read_pdf(df.loc[5,'link_report'], pages='28')

In [210]:
tables[0].df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,11. ACTUAL POWER SUPPLY POSITION - May 2018,,,,,,,,,,,,,,
1,,REGION,STATE,Requirement/Availability in \nMU/DAY,,,Requirement/Availability in MU,,,,Peak Demand/Peak Met in MW,,,,
2,,,,Requirement,Energ\ny met,Surplus(+)/\nDeficit(-),Requirem\nent,Energy \nmet,Surplus(+)/\nDeficit(-),% \nShortage,Requirement,Peak \nDemand \nMet,Surplus(+)/\nDeficit(-),% \nShorta\nge,
3,,NR,Chandigarh,5.2,5.2,0.0,161.5,161.5,0.0,0.0,350,350,0,0.0,
4,,,Delhi,108.6,108.6,0.0,3368.0,3366.8,-1.2,0.0,6346,6346,0,0.0,
5,,,Haryana,150.1,147.7,-2.4,4652.2,4577.5,-74.7,-1.6,8351,8351,0,0.0,
6,,,Himachal Pradesh,25.6,25.3,-0.3,793.8,783.5,-10.3,-1.3,1405,1405,0,0.0,
7,,,J&K,48.4,39.2,-9.2,1500.2,1214.2,-286.0,-19.1,2945,2356,-589,-20.0,
8,,,Punjab,160.5,160.5,0.0,4974.4,4974.4,0.0,0.0,8973,8973,0,0.0,
9,,,Rajasthan,222.1,219.9,-2.2,6885.2,6817.9,-67.3,-1.0,11298,11298,0,0.0,


In [88]:
########################  With camelot Library  ######################f
for k in range(df.shape[0]):
    tables = camelot.read_pdf(df.loc[k,'link_report'], pages='21')
    row = list(tables[0].df.loc[27,2:13]) + [df.iloc[k,0], df.iloc[k,1]]
    print(row,len(row))
    row_dict = {df_.columns[i]:row[i] for i in range(len(row))}
    df_ = df_.append(row_dict, ignore_index=True)

['TNEB', '303.1', '303.0', '-0.1', '9396.1', '9392.7', '-3.4', '0.0', '14613', '14584', '-29', '-0.2', '21-04-2017', 'Monthly Report March 2017'] 14


IncompleteRead: IncompleteRead(294060 bytes read, 2443544 more expected)

## Particular cases done manually (converted to markdown to desactivate)
#for  p,k in {'26' : 8 , '26' : 9 , '21' : 10, '21' : 11}.items():
# [5,6,7,8,9,10,11 ]
for k in [9,10,11 ]:
# in range(3,12):
    tables = camelot.read_pdf(df.loc[k,'link_report'], pages='28')
    row = list(tables[0].df.loc[27,2:13]) + [df.iloc[k,0], df.iloc[k,1]]
    print(row,len(row))
    row_dict = {df_.columns[i]:row[i] for i in range(len(row))}
    df_ = df_.append(row_dict, ignore_index=True)

l=[['Tamil Nadu', '318.3', '318.0', '-0.3', '9868.3', '9858.7', '-9.6', '-0.1', '15251', '15193', '-58', '-0.4', '23-04-2018', 'Monthly Report March 2018'],
['Tamil Nadu', '278.1', '277.9', '-0.2', '8620.9', '8613.9', '-6.9', '-0.1', '14406', '14373', '-33', '-0.2', '23-02-2018', 'Monthly Report January 2018'],
['TNEB', '269.5', '269.2', '-0.3', '8085.2', '8076.0', '-9.2', '-0.1', '14561', '14561', '0', '0.0', '22-12-2017', 'Monthly Report November 2017'] ]

for row in l:
    row_dict = {df_.columns[i]:row[i] for i in range(len(row))}
    df_ = df_.append(row_dict, ignore_index=True)

l=[['TNEB', '257', '198', '-59', '7707', '5940', '-1767', '-23', '11661', '10078', '-1583', '-14', '21-07-2014', 'Monthly Report April 2012'] ]

for row in l:
    row_dict = {df_.columns[i]:row[i] for i in range(len(row))}
    df_ = df_.append(row_dict, ignore_index=True)

In [ ]:
########################  With tabula Library  ######################
for k in list(range(df.shape[0])):
#for k in [5,6,7,8,9,10,11]:
    # Extract dataframe
    dfs = tabula.read_pdf(df.loc[k,'link_report'], lattice=True, pandas_options={"header": [0, 1]},  pages='27')
    
    row = list(dfs[0].loc[28,2:13]) + [df.iloc[k,0], df.iloc[k,1]]
    print(row,len(row))
    row_dict = {df_.columns[i]:row[i] for i in range(len(row))}
    df_ = df_.append(row_dict, ignore_index=True)

KeyError: 28

In [193]:
df_

,state,Requirement(MU/DAY),Energy_met(MU/DAY),Surplus(+)/Deficit(-)(MU/DAY),Requirement(MU),Energy_met(MU),Surplus(+)/Deficit(-)(MU),%Shortage(MU),Requirement(MW),Peak_Demand_Met5(MW),Surplus(+)/Deficit(-)(MW),%Shortage(MW),date_report,name_report
0,Tamil Nadu,354,354,-0.9,10632,10605,-26,-0.2,17646,17563,-83,-0.5,23-05-2022,Monthly_Report_Apr_2022
1,Tamil Nadu,337,337,0,10459,10458,-1,0,16906,16906,0,0,23-06-2022,Monthly_Report_May_2022
2,Tamil Nadu,344,343,-0.3,10657,10648,-9,-0.1,17196,17196,0,0,25-04-2022,Monthly_Report_Mar_2022
3,Tamil Nadu,286,286,0,8873,8873,1,0,15290,15290,0,0,23-02-2022,Monthly_Report_Jan_2022
4,Tamil Nadu,245,245,0,7362,7360,-1,0,13501,13480,-21,-0.2,23-12-2021,Monthly_Report_Nov_2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,TNEB,264,230,-35,7928,6893,-1035,-13,12548,11283,-1265,-10,21-07-2014,June 2012 Monthly Report
120,TNEB,254,222,-32,7605,6649,-956,-13,10968,10491,-477,-4,21-07-2014,May 2012 Monthly Report
121,TNEB,1881,1977,96,63,66,3,,21-07-2014,April 2012 Monthly Report,NaN,NaN,NaN,NaN
122,TNEB,257,198,-59,7707,5940,-1767,-23,11661,10078,-1583,-14,21-07-2014,Monthly Report April 2012


In [213]:
df_.to_csv('loadschedding_TamilNadu.csv', index=False)


In [214]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   state                          130 non-null    object
 1   Requirement(MU/DAY)            130 non-null    object
 2   Energy_met(MU/DAY)             130 non-null    object
 3   Surplus(+)/Deficit(-)(MU/DAY)  130 non-null    object
 4   Requirement(MU)                130 non-null    object
 5   Energy_met(MU)                 130 non-null    object
 6   Surplus(+)/Deficit(-)(MU)      130 non-null    object
 7   %Shortage(MU)                  130 non-null    object
 8   Requirement(MW)                130 non-null    object
 9   Peak_Demand_Met5(MW)           130 non-null    object
 10  Surplus(+)/Deficit(-)(MW)      130 non-null    object
 11  %Shortage(MW)                  130 non-null    object
 12  date_report                    130 non-null    object
 13  name_